In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pygad
import wandb

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

tf.random.set_seed(0)
np.random.seed(0)

2023-06-24 20:42:29.954384: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 20:42:30.919935: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-24 20:42:35.028066: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/macula/SMATousi/.conda/envs/mac-deep/lib/:/home/macula/SMATousi/.conda/envs/mac-deep/lib/py

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5569027851531129930
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 395378688
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11555201948193769516
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2023-06-24 20:42:44.473726: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 20:42:44.545520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-24 20:42:44.652984: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-24 20:42:44.653346: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at

In [8]:
import csv


# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "/home/macula/SMATousi/VAE_outlier/datasets/Musk/musk_csv.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
#         all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
#         all_targets.append([int(fields[-1].replace('"', ""))])
        all_features.append([v.replace('', "") for v in fields[3:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])
            print("EXAMPLE TARGET:", all_targets)

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: ID,molecule_name,conformation_name,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,class
EXAMPLE FEATURES: ['46', '-108', '-60', '-69', '-117', '49', '38', '-161', '-8', '5', '-323', '-220', '-113', '-299', '-283', '-307', '-31', '-106', '-227', '-42', '-59', '-22', '-67', '189', '81', '17', '-27', '-89', '-67', '105

In [12]:
num_rows = features.shape[0]
random_indices = np.random.permutation(num_rows)

features = features[random_indices]
targets = targets[random_indices]

num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 5279
Number of validation samples: 1319


In [16]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 818 (15.50% of total)


In [17]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

In [19]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


latent_dim = 2

encoder_inputs = keras.Input(shape=(166,))
# x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
# x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Flatten()(x)
x = layers.Dense(30, activation="tanh")(encoder_inputs)
x = layers.Dense(20, activation="tanh")(x)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(16, activation="tanh")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

latent_inputs = keras.Input(shape=(latent_dim,))
# x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
# x = layers.Reshape((7, 7, 64))(x)
# x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
# decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
x = layers.Dense(16, activation="tanh")(latent_inputs)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(20, activation="tanh")(x)
decoder_outputs = layers.Dense(166, activation="tanh")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")



class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
#             reconstruction_loss = tf.reduce_mean(
#                 tf.reduce_sum(
#                     keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
#                 )
#             )
            reconstruction_loss = tf.keras.losses.MeanSquaredError()(data,reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

creditdata = np.concatenate([train_features, val_features], axis=0)
creditdata = np.expand_dims(creditdata, -1).astype("float32")

vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.legacy.Adam())
vae.fit(creditdata, epochs=100, batch_size=128)

Epoch 1/100
11/52 [=====>........................] - ETA: 0s - loss: 1.4243 - reconstruction_loss: 1.0129 - kl_loss: 0.2850  

2023-06-24 21:04:09.568600: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


52/52 [==============================] - 1s 5ms/step - loss: 1.2115 - reconstruction_loss: 1.0038 - kl_loss: 0.0890
Epoch 2/100
52/52 [==============================] - 0s 4ms/step - loss: 1.0096 - reconstruction_loss: 1.0004 - kl_loss: 0.0087
Epoch 3/100
52/52 [==============================] - 0s 5ms/step - loss: 1.0011 - reconstruction_loss: 1.0004 - kl_loss: 0.0043
Epoch 4/100
52/52 [==============================] - 0s 5ms/step - loss: 0.9995 - reconstruction_loss: 1.0000 - kl_loss: 0.0027
Epoch 5/100
52/52 [==============================] - 0s 5ms/step - loss: 1.0024 - reconstruction_loss: 1.0001 - kl_loss: 0.0018
Epoch 6/100
52/52 [==============================] - 0s 5ms/step - loss: 0.9942 - reconstruction_loss: 0.9997 - kl_loss: 0.0014
Epoch 7/100
52/52 [==============================] - 0s 6ms/step - loss: 1.0019 - reconstruction_loss: 1.0006 - kl_loss: 0.0011
Epoch 8/100
52/52 [==============================] - 0s 5ms/step - loss: 1.0007 - reconstruction_loss: 0.9997 - kl_l

Epoch 63/100
52/52 [==============================] - 0s 5ms/step - loss: 1.0053 - reconstruction_loss: 0.9999 - kl_loss: 2.0941e-05
Epoch 64/100
52/52 [==============================] - 0s 5ms/step - loss: 0.9984 - reconstruction_loss: 1.0004 - kl_loss: 2.0604e-05
Epoch 65/100
52/52 [==============================] - 0s 5ms/step - loss: 1.0009 - reconstruction_loss: 1.0000 - kl_loss: 2.5035e-05
Epoch 66/100
52/52 [==============================] - 0s 6ms/step - loss: 1.0012 - reconstruction_loss: 1.0001 - kl_loss: 2.0266e-05
Epoch 67/100
52/52 [==============================] - 0s 5ms/step - loss: 1.0012 - reconstruction_loss: 1.0001 - kl_loss: 1.8495e-05
Epoch 68/100
52/52 [==============================] - 0s 5ms/step - loss: 1.0033 - reconstruction_loss: 1.0000 - kl_loss: 1.8668e-05
Epoch 69/100
52/52 [==============================] - 0s 5ms/step - loss: 1.0013 - reconstruction_loss: 0.9993 - kl_loss: 1.7143e-05
Epoch 70/100
52/52 [==============================] - 0s 4ms/step - l

In [22]:
rec_loss = []

for i in range(train_features.shape[0]):
    
    if train_targets[i] == 0:
        
        test = train_features[i,:]

        test = test.reshape([1,166])

        z_mean, z_log_var, z = vae.encoder(test)
        reconstruction = vae.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(test,reconstruction)
        
        rec_loss.append(reconstruction_loss.numpy())


        print("Rec Loss = ", reconstruction_loss.numpy(), " -------- Label = ", train_targets[i], " Case No. ", i)

rec_loss = np.array(rec_loss)
print("mean = ", np.mean(rec_loss))
print("std = ", np.std(rec_loss))

Rec Loss =  0.6612147  -------- Label =  [0]  Case No.  0
Rec Loss =  0.9504867  -------- Label =  [0]  Case No.  2
Rec Loss =  0.6104503  -------- Label =  [0]  Case No.  3
Rec Loss =  0.9403547  -------- Label =  [0]  Case No.  4
Rec Loss =  1.022213  -------- Label =  [0]  Case No.  5
Rec Loss =  0.673464  -------- Label =  [0]  Case No.  6
Rec Loss =  0.5424545  -------- Label =  [0]  Case No.  7
Rec Loss =  1.076164  -------- Label =  [0]  Case No.  8
Rec Loss =  1.1022438  -------- Label =  [0]  Case No.  9
Rec Loss =  1.4458702  -------- Label =  [0]  Case No.  10
Rec Loss =  0.8873674  -------- Label =  [0]  Case No.  11
Rec Loss =  1.0643046  -------- Label =  [0]  Case No.  12
Rec Loss =  1.4083717  -------- Label =  [0]  Case No.  13
Rec Loss =  0.7557866  -------- Label =  [0]  Case No.  14
Rec Loss =  0.6797641  -------- Label =  [0]  Case No.  15
Rec Loss =  1.3422731  -------- Label =  [0]  Case No.  17
Rec Loss =  0.9964783  -------- Label =  [0]  Case No.  18
Rec Loss 

Rec Loss =  0.91024625  -------- Label =  [0]  Case No.  241
Rec Loss =  1.0946637  -------- Label =  [0]  Case No.  242
Rec Loss =  0.7202847  -------- Label =  [0]  Case No.  243
Rec Loss =  0.6762055  -------- Label =  [0]  Case No.  244
Rec Loss =  0.73608005  -------- Label =  [0]  Case No.  245
Rec Loss =  1.0396103  -------- Label =  [0]  Case No.  246
Rec Loss =  0.7288142  -------- Label =  [0]  Case No.  247
Rec Loss =  1.3395641  -------- Label =  [0]  Case No.  248
Rec Loss =  1.1212054  -------- Label =  [0]  Case No.  249
Rec Loss =  1.2607918  -------- Label =  [0]  Case No.  250
Rec Loss =  1.1241614  -------- Label =  [0]  Case No.  251
Rec Loss =  1.0860665  -------- Label =  [0]  Case No.  252
Rec Loss =  1.1918164  -------- Label =  [0]  Case No.  253
Rec Loss =  1.0099863  -------- Label =  [0]  Case No.  254
Rec Loss =  1.0156465  -------- Label =  [0]  Case No.  256
Rec Loss =  0.9535551  -------- Label =  [0]  Case No.  257
Rec Loss =  1.5777367  -------- Label 

Rec Loss =  1.4358699  -------- Label =  [0]  Case No.  481
Rec Loss =  1.0089824  -------- Label =  [0]  Case No.  482
Rec Loss =  1.1504971  -------- Label =  [0]  Case No.  483
Rec Loss =  0.9772872  -------- Label =  [0]  Case No.  485
Rec Loss =  0.92325747  -------- Label =  [0]  Case No.  486
Rec Loss =  1.087419  -------- Label =  [0]  Case No.  488
Rec Loss =  0.80167764  -------- Label =  [0]  Case No.  489
Rec Loss =  0.8861923  -------- Label =  [0]  Case No.  490
Rec Loss =  0.9748777  -------- Label =  [0]  Case No.  491
Rec Loss =  1.2749957  -------- Label =  [0]  Case No.  492
Rec Loss =  0.7077461  -------- Label =  [0]  Case No.  493
Rec Loss =  0.8611779  -------- Label =  [0]  Case No.  494
Rec Loss =  0.99669635  -------- Label =  [0]  Case No.  496
Rec Loss =  1.0339262  -------- Label =  [0]  Case No.  497
Rec Loss =  0.97674716  -------- Label =  [0]  Case No.  498
Rec Loss =  0.9822774  -------- Label =  [0]  Case No.  499
Rec Loss =  0.45496836  -------- Labe

Rec Loss =  0.9478636  -------- Label =  [0]  Case No.  711
Rec Loss =  0.74391234  -------- Label =  [0]  Case No.  712
Rec Loss =  1.1323938  -------- Label =  [0]  Case No.  713
Rec Loss =  0.6604362  -------- Label =  [0]  Case No.  714
Rec Loss =  0.63282984  -------- Label =  [0]  Case No.  715
Rec Loss =  0.8557653  -------- Label =  [0]  Case No.  718
Rec Loss =  1.0193563  -------- Label =  [0]  Case No.  719
Rec Loss =  1.0383892  -------- Label =  [0]  Case No.  721
Rec Loss =  1.3035654  -------- Label =  [0]  Case No.  723
Rec Loss =  0.94984996  -------- Label =  [0]  Case No.  724
Rec Loss =  1.0698718  -------- Label =  [0]  Case No.  725
Rec Loss =  0.67203814  -------- Label =  [0]  Case No.  726
Rec Loss =  0.77143776  -------- Label =  [0]  Case No.  727
Rec Loss =  0.8666689  -------- Label =  [0]  Case No.  728
Rec Loss =  0.90873075  -------- Label =  [0]  Case No.  730
Rec Loss =  1.2000881  -------- Label =  [0]  Case No.  732
Rec Loss =  0.77578396  -------- L

Rec Loss =  0.95083904  -------- Label =  [0]  Case No.  944
Rec Loss =  1.3972775  -------- Label =  [0]  Case No.  945
Rec Loss =  0.8594172  -------- Label =  [0]  Case No.  946
Rec Loss =  0.97391915  -------- Label =  [0]  Case No.  947
Rec Loss =  1.1557599  -------- Label =  [0]  Case No.  948
Rec Loss =  1.4195503  -------- Label =  [0]  Case No.  949
Rec Loss =  0.9323141  -------- Label =  [0]  Case No.  950
Rec Loss =  1.3002992  -------- Label =  [0]  Case No.  952
Rec Loss =  1.3680145  -------- Label =  [0]  Case No.  953
Rec Loss =  0.8602815  -------- Label =  [0]  Case No.  954
Rec Loss =  0.89014965  -------- Label =  [0]  Case No.  955
Rec Loss =  1.0080255  -------- Label =  [0]  Case No.  956
Rec Loss =  1.3889693  -------- Label =  [0]  Case No.  958
Rec Loss =  0.9801531  -------- Label =  [0]  Case No.  960
Rec Loss =  1.417051  -------- Label =  [0]  Case No.  961
Rec Loss =  0.7926995  -------- Label =  [0]  Case No.  963
Rec Loss =  1.1216047  -------- Label 

...

Rec Loss =  0.5794239  -------- Label =  [0]  Case No.  4624
Rec Loss =  0.9035875  -------- Label =  [0]  Case No.  4625
Rec Loss =  1.3527185  -------- Label =  [0]  Case No.  4626
Rec Loss =  0.9593394  -------- Label =  [0]  Case No.  4627
Rec Loss =  1.1245915  -------- Label =  [0]  Case No.  4628
Rec Loss =  0.60826445  -------- Label =  [0]  Case No.  4629
Rec Loss =  0.62963694  -------- Label =  [0]  Case No.  4630
Rec Loss =  0.7615888  -------- Label =  [0]  Case No.  4631
Rec Loss =  0.6749851  -------- Label =  [0]  Case No.  4632
Rec Loss =  0.7699606  -------- Label =  [0]  Case No.  4633
Rec Loss =  0.84193236  -------- Label =  [0]  Case No.  4634
Rec Loss =  0.9905665  -------- Label =  [0]  Case No.  4635
Rec Loss =  1.4124284  -------- Label =  [0]  Case No.  4637
Rec Loss =  1.4228574  -------- Label =  [0]  Case No.  4638
Rec Loss =  0.82760286  -------- Label =  [0]  Case No.  4639
Rec Loss =  0.7170704  -------- Label =  [0]  Case No.  4640
Rec Loss =  0.806043

Rec Loss =  1.1097859  -------- Label =  [0]  Case No.  4783
Rec Loss =  1.252327  -------- Label =  [0]  Case No.  4784
Rec Loss =  0.73132455  -------- Label =  [0]  Case No.  4785
Rec Loss =  0.9692773  -------- Label =  [0]  Case No.  4786
Rec Loss =  0.8510137  -------- Label =  [0]  Case No.  4787
Rec Loss =  0.88494045  -------- Label =  [0]  Case No.  4788
Rec Loss =  0.795835  -------- Label =  [0]  Case No.  4789
Rec Loss =  1.0222569  -------- Label =  [0]  Case No.  4790
Rec Loss =  1.375814  -------- Label =  [0]  Case No.  4791
Rec Loss =  1.0291376  -------- Label =  [0]  Case No.  4792
Rec Loss =  0.9162096  -------- Label =  [0]  Case No.  4793
Rec Loss =  1.4201888  -------- Label =  [0]  Case No.  4794
Rec Loss =  0.9449248  -------- Label =  [0]  Case No.  4795
Rec Loss =  0.70647407  -------- Label =  [0]  Case No.  4797
Rec Loss =  1.2714858  -------- Label =  [0]  Case No.  4798
Rec Loss =  1.1082376  -------- Label =  [0]  Case No.  4799
Rec Loss =  1.0811232  -

Rec Loss =  0.8355961  -------- Label =  [0]  Case No.  4939
Rec Loss =  1.6220703  -------- Label =  [0]  Case No.  4940
Rec Loss =  1.3937135  -------- Label =  [0]  Case No.  4942
Rec Loss =  0.7635076  -------- Label =  [0]  Case No.  4943
Rec Loss =  1.2948084  -------- Label =  [0]  Case No.  4944
Rec Loss =  0.8681842  -------- Label =  [0]  Case No.  4945
Rec Loss =  0.82134855  -------- Label =  [0]  Case No.  4946
Rec Loss =  1.6116583  -------- Label =  [0]  Case No.  4947
Rec Loss =  1.6741221  -------- Label =  [0]  Case No.  4948
Rec Loss =  1.3046433  -------- Label =  [0]  Case No.  4949
Rec Loss =  0.75461185  -------- Label =  [0]  Case No.  4950
Rec Loss =  1.192297  -------- Label =  [0]  Case No.  4951
Rec Loss =  0.84380454  -------- Label =  [0]  Case No.  4952
Rec Loss =  1.3171909  -------- Label =  [0]  Case No.  4953
Rec Loss =  1.6881869  -------- Label =  [0]  Case No.  4954
Rec Loss =  1.2430363  -------- Label =  [0]  Case No.  4956
Rec Loss =  1.904656  

Rec Loss =  1.2437394  -------- Label =  [0]  Case No.  5098
Rec Loss =  0.689369  -------- Label =  [0]  Case No.  5099
Rec Loss =  0.87945455  -------- Label =  [0]  Case No.  5100
Rec Loss =  1.1562146  -------- Label =  [0]  Case No.  5101
Rec Loss =  0.66783446  -------- Label =  [0]  Case No.  5104
Rec Loss =  0.90241313  -------- Label =  [0]  Case No.  5105
Rec Loss =  0.74699193  -------- Label =  [0]  Case No.  5107
Rec Loss =  0.6609766  -------- Label =  [0]  Case No.  5108
Rec Loss =  0.946038  -------- Label =  [0]  Case No.  5109
Rec Loss =  1.5037751  -------- Label =  [0]  Case No.  5111
Rec Loss =  0.8774485  -------- Label =  [0]  Case No.  5113
Rec Loss =  1.5311995  -------- Label =  [0]  Case No.  5114
Rec Loss =  0.64714116  -------- Label =  [0]  Case No.  5115
Rec Loss =  0.96151495  -------- Label =  [0]  Case No.  5116
Rec Loss =  1.1927347  -------- Label =  [0]  Case No.  5117
Rec Loss =  1.188962  -------- Label =  [0]  Case No.  5119
Rec Loss =  0.5446179

Rec Loss =  0.94090456  -------- Label =  [0]  Case No.  5257
Rec Loss =  1.1943754  -------- Label =  [0]  Case No.  5258
Rec Loss =  1.48556  -------- Label =  [0]  Case No.  5259
Rec Loss =  1.372036  -------- Label =  [0]  Case No.  5261
Rec Loss =  1.4585649  -------- Label =  [0]  Case No.  5262
Rec Loss =  0.6109794  -------- Label =  [0]  Case No.  5263
Rec Loss =  1.3926361  -------- Label =  [0]  Case No.  5264
Rec Loss =  0.7339619  -------- Label =  [0]  Case No.  5265
Rec Loss =  0.73141545  -------- Label =  [0]  Case No.  5266
Rec Loss =  0.7596408  -------- Label =  [0]  Case No.  5268
Rec Loss =  0.9809424  -------- Label =  [0]  Case No.  5269
Rec Loss =  1.079875  -------- Label =  [0]  Case No.  5270
Rec Loss =  0.74012804  -------- Label =  [0]  Case No.  5271
Rec Loss =  0.8384802  -------- Label =  [0]  Case No.  5272
Rec Loss =  0.8292094  -------- Label =  [0]  Case No.  5273
Rec Loss =  0.8143124  -------- Label =  [0]  Case No.  5274
Rec Loss =  0.7499084  --